# Projeto Final - Tera - Data Science
Notebooks destinados ao desenvolvimento do projeto final do treinamento Tera - Data Science
## Clusterização
1. Objetivo:
    1. Clusterizar as lojas por similaridade 
    2. Identificar os benchmarks de cada cluster
    3. Calcular as lacunas de acordo com os benchmarks
2. Output:
    1. DataSet com as lojas clusterizadas
    2. Lacunas calculadas

In [1]:
# Bibliotecas Padrões
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime